### This is an example of creating a skeleton file which can be used as a starting point to https://github.com/elwazi/health_data_transformation

In [1]:
import pandas as pd
import numpy as np
import fsspec

In [2]:
fs = fsspec.filesystem("")

Get target codebook

In [3]:
target_df = pd.read_csv('../app/input/target_variables.csv')

In [4]:
target_df

,variable_name,description
0,date,Date
1,time (local),Time
2,dob,Birth Date
3,country,Country
4,patient_id,Patient Identifier
...,...,...
138,diastolic_blood_pressure,Diastolic Blood Pressure
139,mean_arterial_pressure,Mean Arterial Pressure
140,oxygen_saturation,Oxygen Saturation Measurement
141,stroke,Stroke


Get studies that have been mapped:

In [5]:
avail_studies = fs.ls('../app/results/')
avail_studies = [f.split('/')[-1] for f in avail_studies if f.split('/')[-1][0] != '.']
avail_studies

['H3Africa.csv', 'Europe_CH_SIB.csv']

Open a study mapping file that has been generated:

In [6]:
study_file = avail_studies[1]
study = study_file.split('.')[0]

In [7]:
study_df = pd.read_csv(f'../app/results/{study_file}')

In [8]:
study_variables = pd.read_csv(f'../app/input/{study}/dataset_variables.csv')

In [9]:
study_variables

,Unnamed: 0,variable_name,description
0,0,pt,NaN
1,1,phyact,NaN
2,2,alcfrq,NaN
3,3,sbsmk,NaN
4,4,ethori_self,NaN
5,5,jobtyp,NaN
6,6,lvpl,NaN
7,7,SBP,NaN
8,8,DBP,NaN
9,9,mrtsts2,NaN


In [10]:
mapped_df = study_df[study_df['marked'] == 'Successfully mapped']

In [11]:
mapped_df

,study_var,codebook_var,confidence,notes,marked
14,gender,Sex,88%,NaN,Successfully mapped
15,age,Age-Years,88%,NaN,Successfully mapped
16,pt,Patient Identifier,77%,NaN,Successfully mapped
17,ht,Height,87%,NaN,Successfully mapped
18,HRTRTE,Heart Rate,82%,NaN,Successfully mapped
19,wt,Body Weight,78%,NaN,Successfully mapped


In [12]:
# Define the column names
columns = [
    'Study Variable Description',
    'Study Variable Name',
    'Study Variable Coding',
    'Study Variable Format',
    'NEW Variable Name',
    'NEW Variable Description',
    'NEW Variable Coding',
    'NEW Variable Format',
    'NEW to Study Mapping'
]

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

df['Study Variable Name'] = mapped_df.study_var
df['Study Variable Description'] = [study_variables[study_variables.variable_name == x].description.values[0] for x in mapped_df.study_var]
df['NEW Variable Description'] = mapped_df.codebook_var
df['NEW Variable Name'] = [target_df[target_df.description == x[:-1]].variable_name.values[0] for x in mapped_df.codebook_var]

In [13]:
example_data = pd.read_csv(f'../app/input/{study}/example_data.csv')

In [17]:
var_coding = []
for col_name in mapped_df.study_var:
    data = example_data[col_name].dropna()
    if len(data) == 10:
        out.append('')
        var_coding.append(None)
    else:
        out = []
        for v in data:
            out.append(f'{v}, ?')
        var_coding.append('| '.join(out))

df['Study Variable Coding'] = var_coding

In [23]:
df

,Study Variable Description,Study Variable Name,Study Variable Coding,Study Variable Format,NEW Variable Name,NEW Variable Description,NEW Variable Coding,NEW Variable Format,NEW to Study Mapping
14,NaN,gender,"0.0, ?| 1.0, ?",NaN,sex,Sex,NaN,NaN,gender = sex
15,NaN,age,None,NaN,age_enrolment,Age-Years,NaN,NaN,age = age_enrolment
16,NaN,pt,None,NaN,patient_id,Patient Identifier,NaN,NaN,pt = patient_id
17,NaN,ht,None,NaN,height,Height,NaN,NaN,ht = height
18,NaN,HRTRTE,None,NaN,heart_rate,Heart Rate,NaN,NaN,HRTRTE = heart_rate
19,NaN,wt,None,NaN,weight,Body Weight,NaN,NaN,wt = weight


In [24]:
df['NEW to Study Mapping'] = [f'{new} = {old}' for old,new in zip(df['Study Variable Name'], df['NEW Variable Name'])]

In [26]:
df.to_excel('to_transformation.xlsx')